In [ ]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

# 필요한 함수들을 불러줍니다!
import re
import time
from pandas import DataFrame
import pandas as pd
import numpy as np

In [ ]:
# 크롤링 데이터가 저장될 데이터프레임 만들기
total_info = pd.DataFrame(columns=['이름', '소분류', '주소', '별점', '리뷰 수', '리뷰'])
total_info

In [ ]:
# End-To-End 도전!


# 기본 설정하기

# 크롬드라이버 경로 설정
# 경로는 본인의 chromedriver 경로로 설정
path = "C:/Users/rnjst/OneDrive/바탕 화면/딥러닝개론/Final Project/chromedriver"   # 여기 수정!!

# 드라이버 객체 생성
browser = webdriver.Chrome(path)

# publicdata_allcleared.csv 파일을 불러옵니다! 디렉토리 유의하세요!
df = pd.read_csv('C:/Users/rnjst/OneDrive/바탕 화면/딥러닝개론/Final Project/publicdata_allcleared.csv')   # 여기 수정!

# 네이버 지도 주소 입력
browser.get('https://map.naver.com/v5/')
time.sleep(2)

In [ ]:
# 시간 측정
start = time.time()

for num_iter in range(0, 100): # 크롤링할 영역에 맞게 수정하세요!
    
    # 음식점의 이름을 name으로 저장
    name = df.loc[num_iter, '상호명']
    
    # 행복한 이스터에그
    if num_iter % 10 == 0:
        print('와! {0}개째 Crawling!'.format(num_iter))
        
    # 음식점의 법정동명을 real_location으로 저장
    real_location = df.loc[num_iter, '법정동명']
    
    # 검색창에 접속하기
    search = browser.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    
    # 검색창에 검색어 넣기
    search.clear() # 검색어창 청소
    search.send_keys(name)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    time.sleep(2)
    
    # 검색결과에 맞춰서 음식점 창이 딱 뜬 경우
    try:
        frame = browser.find_element(By.CSS_SELECTOR, "iframe#entryIframe")
        browser.switch_to.frame(frame)
    
    # 음식점 창이 바로 안 뜬 경우
    except:
        # searchIframe으로 변경
        frame = browser.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
        browser.switch_to.frame(frame)
                                        
        
        try:
            # 데이터에서 가져온 법정동명과 검색해서 나온 음식점 주소 비교
            isnt_location = True
            order = 0
            while isnt_location:
                order += 1
                try: # 리스트에 음식점이 2개 이상인 경우
                    location = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[{0}]/div[2]/div'.format(order)).text
                
                except: # 리스트에 음식점이 1개인 경우
                    location = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/div[1]/div/span/a/span[1]').text
                
                # 음식점 주소와 법정동명 일치하면, 바로 그 음식점이 우리가 찾는 음식점?
                if real_location in location:
                    isnt_location = False
                
                else:
                    # 최대 탐색 개수는 15로 제한
                    if order == 15:
                        browser.switch_to.parent_frame()
                        continue
                 
            # 링크 버튼도 음식점 개수에 따라 XPATH가 다릅니다
            try:
                link_button = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[{0}]/div[2]/a[1]'.format(order))
            except:
                link_button = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/a')
            
            link_button.click()
            time.sleep(1)
    
            browser.switch_to.parent_frame()
        
            frame = browser.find_element(By.CSS_SELECTOR, 'iframe#entryIframe')
            browser.switch_to.frame(frame)
    
        # 아니면 검색된 링크가 없는 거다
        except:
            print('{0}번 음식점 {1}, 이 곳은 망한 것 같습니다.'.format(num_iter, name))
            browser.switch_to.parent_frame()
            continue
        
    
    # 이 아래부분부터 해당 음식점이 존재하는 경우에만 돌아가게 됩니다!   
    # 식당 이름, 소분류, 주소, 별점 크롤링
    total_info.loc[num_iter, '이름'] = name
    try:
        total_info.loc[num_iter, '소분류'] = browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text
    except:
        pass
    try:
        total_info.loc[num_iter, '주소'] = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[1]/div/a/span[1]').text
    except:
        pass
    try:
        total_info.loc[num_iter, '별점'] = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
    except:
        pass
    
    # '리뷰' 버튼 클릭하기!
    # 리뷰가 몇 번째에 있을지가 불확실해서
    # 그 부분을 고려하는 내용입니다
    
    # '리뷰' 버튼이 없으면 아예 넘어가 버리도록 코드 수정!
    finding = True
    where = 1
    try:
        while finding:
            where_is_review = '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{0}]'.format(where)

            please_review = browser.find_element(By.XPATH, value=where_is_review).text
            if please_review != '리뷰':
                where = where + 1

            else:
                link = browser.find_element(By.XPATH, value=where_is_review).get_attribute('href')
                finding = False
            
    except:
        print('{0}번 음식점 {1} 마음에 안 들어요. 이 오류 수정하는 데만 1시간 걸렸거든요. 다음!'.format(num_iter, name))
        browser.switch_to.parent_frame()
        continue
        
    # 이 링크를 새로운 창으로 열겠습니다.
    # 그 과정을 볼게요.

    # 탭을 하나 더 열겠습니다. 그냥 구글으로 열었어요.
    browser.execute_script('window.open("https://google.com");')

    # 구글으로 열리기까지 시간을 좀 기다려줄게요.
    time.sleep(1)

    # 기존에 주피터가 조종하던 창은, 지도가 있고 프레임이 있는 탭이었는데
    # 이제 새로 연 탭을 주피터가 조종하도록 지시해줄게요.
    browser.switch_to.window(browser.window_handles[-1])

    # 새로연 탭을 조종할 수 있도록 했으니 이제 그 탭에서 저희가 가지고 있는 리뷰 링크를 열겠습니다.
    browser.get(link)
    time.sleep(2)
    
    # 식당 리뷰 수 크롤링
    try:
        total_info.loc[num_iter, '리뷰 수'] = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[3]/h2/span[1]').text
    except:
        print('{0}번 음식점 {1}, 리뷰가 부족해요'.format(num_iter, name))
        browser.close()
        browser.switch_to.window(browser.window_handles[0])        
        continue
    
    # 식당 리뷰를 저장할 리스트 생성
    review = []
    
    for i in range(1, int(total_info.loc[num_iter, '리뷰 수'].replace(',',''))+1):
        try: # 사진 리뷰 있을 때 xpath
            review_text = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[3]/div/ul/li['+str(i)+']/div[3]/a/span').text
            if review_text == '개의 리뷰가 더 있습니다': # 사진 리뷰 없을 때 위의 xpath 사용하면 '개의 리뷰가 더 있습니다' 뜨는 경우 발생
                review_text = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[3]/div/ul/li['+str(i)+']/div[2]/a/span').text
            review.append(review_text)
        except:
            try: # 사진 리뷰 없을 때 xpath
                review_text = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[3]/div/ul/li['+str(i)+']/div[2]/a/span').text
                review.append(review_text)
            except: # 텍스트 리뷰가 없을 때 그냥 pass 시키기
                pass
            
        if i % 10 == 0: # 왜 식당마다 더보기 xpath가 다른거죠? ;;
            try:
                browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[3]/div[2]/a').click()
            except:
                try:
                    browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[2]/div[2]/a').click()
                except:
                    pass
            time.sleep(2)
    
    # 데이터프레임 안에 리스트를 못 집어넣어서 리스트를 string으로 바꿔서 집어넣기
    # 나중에 사용할 때는 total_info.loc[num_iter, '리뷰'][2:-2].split("', '") 코드 사용
    total_info.loc[num_iter, '리뷰'] = str(review)
    
    # 한 주기 마치기
    browser.close()
    browser.switch_to.window(browser.window_handles[0])

# 종료
# browser.quit()

end = time.time()
# 걸린 시간
print(f"{end - start:.5f} sec")

In [ ]:
print(len(total_info))
total_info.tail()

In [ ]:
total_info.to_csv('C:/Users/rnjst/OneDrive/바탕 화면/딥러닝개론/Final Project/total_info.csv', encoding='utf-8-sig')